In [1]:
user_input = "What is Machine vision detection to daily facial fatigue ?"
CATEGORY=''

In [2]:
from config import *
from qdrant_helper import insert_qdrant_docs, \
    get_search_results
import qdrant_client as qc
import qdrant_client.http.models as qmodels
from qdrant_client.http.models import *
from model_helper import load_model
from pprint import pprint

/home/azureuser/Ambarish/env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def get_qdrant_client():
    client = qc.QdrantClient(url=URL)
    METRIC = qmodels.Distance.COSINE
    return client


client = get_qdrant_client()

In [5]:
model = load_model()
xq = model.encode(user_input,convert_to_tensor=True)
if CATEGORY == '':
    query_filter = None
else:
    query_filter=qmodels.Filter(
        must= [
            FieldCondition(
                key="Category",
                match=models.MatchValue(value=CATEGORY),
            )
        ],
    )
search_result = client.search(collection_name=COLLECTION_NAME,
                                query_vector=xq.tolist(), 
                                query_filter=query_filter,
                                limit=3)
contexts =""
for result in search_result:
    contexts +=  result.payload['abstract']+"\n---\n"

In [6]:
pprint(contexts)

('  Fatigue detection is valued for people to keep mental health and prevent\n'
 'safety accidents. However, detecting facial fatigue, especially mild fatigue '
 'in\n'
 'the real world via machine vision is still a challenging issue due to lack '
 'of\n'
 'non-lab dataset and well-defined algorithms. In order to improve the '
 'detection\n'
 'capability on facial fatigue that can be used widely in daily life, this '
 'paper\n'
 'provided an audiovisual dataset named DLFD (daily-life fatigue dataset) '
 'which\n'
 "reflected people's facial fatigue state in the wild. A framework using\n"
 '3D-ResNet along with non-local attention mechanism was training for '
 'extraction\n'
 'of local and long-range features in spatial and temporal dimensions. Then, '
 'a\n'
 'compacted loss function combining mean squared error and cross-entropy was\n'
 'designed to predict both continuous and categorical fatigue degrees. Our\n'
 'proposed framework has reached an average accuracy of 90.8% on validati

In [7]:
import boto3

In [8]:
import os

In [9]:
boto3_bedrock = boto3.client(service_name='bedrock-runtime',
                             region_name='us-east-1')

In [10]:
def create_prompt(context,query):
    
    context2 = f"""
Human: {context}  {query} 
Assistant:"""
    return  context2 

In [11]:
prompt_data = create_prompt(contexts,user_input)  

In [12]:
prompt_data

'\nHuman:   Fatigue detection is valued for people to keep mental health and prevent\nsafety accidents. However, detecting facial fatigue, especially mild fatigue in\nthe real world via machine vision is still a challenging issue due to lack of\nnon-lab dataset and well-defined algorithms. In order to improve the detection\ncapability on facial fatigue that can be used widely in daily life, this paper\nprovided an audiovisual dataset named DLFD (daily-life fatigue dataset) which\nreflected people\'s facial fatigue state in the wild. A framework using\n3D-ResNet along with non-local attention mechanism was training for extraction\nof local and long-range features in spatial and temporal dimensions. Then, a\ncompacted loss function combining mean squared error and cross-entropy was\ndesigned to predict both continuous and categorical fatigue degrees. Our\nproposed framework has reached an average accuracy of 90.8% on validation set\nand 72.5% on test set for binary classification, standi

In [13]:
import json

In [14]:
body = json.dumps({"prompt": prompt_data, "max_tokens_to_sample": 500})
modelId = "anthropic.claude-instant-v1"  
accept = "application/json"
contentType = "application/json"

response = boto3_bedrock.invoke_model(
    body=body, modelId=modelId, accept=accept, contentType=contentType
)
response_body = json.loads(response.get("body").read())


In [16]:
pprint(response_body["completion"])

(' Machine vision detection of daily facial fatigue refers to the use of '
 'computer vision and machine learning techniques to automatically detect and '
 'assess levels of fatigue from facial images of people in daily, real-world '
 'settings. Some key points:\n'
 '\n'
 '- The goal is to detect more mild or subtle forms of fatigue that may occur '
 'during everyday activities, as opposed to obvious fatigue measurable in '
 'controlled lab settings. This is more challenging due to uncontrolled '
 'variables.\n'
 '\n'
 '- Facial fatigue is assessed based on subtle changes in facial features, '
 'expressions, eye gaze, etc. over time that may indicate increasing tiredness '
 'or exhaustion. \n'
 '\n'
 '- Deep learning models, especially convolutional neural networks, are '
 'commonly used to analyze facial images and videos to learn the visual cues '
 'associated with different fatigue levels.\n'
 '\n'
 '- Spatial features within individual frames as well as temporal changes '
 'across 